In [1]:
from configuration import EC2_CONFIG, IAM_CONFIG, PROFILE, CODE_DEPLOY_CONFIG, IAM_CONFIG, S3_CONFIG, STS_CONFIG, CLOUD_WATCH_CONFIG, ECS_CONFIG
from configuration import REGION, AVAILABILITY_ZONE
from aws_service_utils import create_aws_service
from ec2_utils import create_vpc, create_subnet, create_security_group, create_key_pair, \
    lunch_ec2_instance, wait_instances_lunch, get_subnet_ids, start_ec2_instance, stop_ec2_instance

from iam_utils import get_role
from code_deploy_utils import create_application, create_deployment, create_deployment_group

from sts_utils import get_aws_user_account
from s3_utils import create_bucket, put_bucket_policy, upload_server_app_to_s3_bucket


## ecs utils 


In [6]:
import boto3
import base64

import subprocess
ecs_client = boto3.client('ecr', region_name=REGION)
import botocore.exceptions

# Create an ECS repository

repository_name = 'my-ecs-repoo'

try:
    response = ecs_client.create_repository(repositoryName=repository_name)
    print(f"Created ECS repository: {repository_name}")
except botocore.exceptions.ClientError as e:
    if e.response['Error']['Code'] == 'RepositoryAlreadyExistsException':
        print(f"ECS repository {repository_name} already exists.")
    else:
        print(f"Error creating ECS repository: {e}")
        exit(1)







Created ECS repository: my-ecs-repoo



usage: aws [options] <command> <subcommand> [<subcommand> ...] [parameters]
To see help text, you can run:

  aws help
  aws <command> help
  aws <command> <subcommand> help

aws: error: the following arguments are required: command



CalledProcessError: Command '['aws', 'ecr', 'get-login-password', '--region', 'us-east-1']' returned non-zero exit status 252.

In [13]:
# Build and push the Docker image
image_name = f'{repository_name}:latest'
dockerfile_path = '/home/altaf/Desktop/AWS_ML_SERVING/worker/dockerfile'  # Replace with the path to your Dockerfile
ecr_repository_uri = response['repository']['repositoryUri']

# Build the Docker image
#subprocess.run(['docker', 'build', '-t', image_name, '-f', dockerfile_path, '.'], check=True)

# Authenticate Docker to the Amazon ECR registry

# Get an authentication token for ECR
token = ecs_client.get_authorization_token()
authorization_data = token['authorizationData'][0]

# Extract the username and password from the base64-encoded token
token_payload = base64.b64decode(authorization_data['authorizationToken']).decode()
username, password = token_payload.split(":")

# Log in to ECR using the obtained credentials
subprocess.run(['docker', 'login', '-u', username, '-p', password, authorization_data['proxyEndpoint']])

# Push the Docker image to the ECR repository
subprocess.run(['docker', 'tag', 'worker-app', ecr_repository_uri], check=True)
subprocess.run(['docker', 'push', ecr_repository_uri], check=True)

print(f"Pushed Docker image to ECR repository: {ecr_repository_uri}")




WARNING! Using --password via the CLI is insecure. Use --password-stdin.


Login Succeeded


WARNING! Your password will be stored unencrypted in /home/altaf/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Error response from daemon: No such image: my-ecs-repoo:latest


CalledProcessError: Command '['docker', 'tag', 'my-ecs-repoo:latest', '175054333530.dkr.ecr.us-east-1.amazonaws.com/my-ecs-repoo']' returned non-zero exit status 1.

In [4]:
import boto3
import os
import subprocess


region_name = REGION

# Initialize the AWS ECS client
ecs_client = boto3.client('ecs', region_name=region_name)

# Create an ECS repository
repository_name = 'my-ecs-repo'
try:
    response = ecs_client.create_repository(repositoryName=repository_name)
    print(f"Created ECS repository: {repository_name}")
except ecs_client.exceptions.RepositoryAlreadyExistsException:
    print(f"ECS repository {repository_name} already exists.")
except Exception as e:
    print(f"Error creating ECS repository: {e}")
    exit(1)

# Build and push the Docker image
image_name = f'{repository_name}:latest'
dockerfile_path = '/home/altaf/Desktop/AWS_ML_SERVING/worker/dockerfile'  # Replace with the path to your Dockerfile
ecs_repository_uri = response['repository']['repositoryUri']

# Build the Docker image
subprocess.run(['docker', 'build', '-t', image_name, '-f', dockerfile_path, '.'], check=True)

# Authenticate Docker to the Amazon ECR registry
subprocess.run(['aws', 'ecr', 'get-login-password', '--region', region_name], stdout=subprocess.PIPE, text=True, check=True,  shell=True)

# Push the Docker image to the ECS repository
subprocess.run(['docker', 'tag', image_name, ecs_repository_uri], check=True)
subprocess.run(['docker', 'push', ecs_repository_uri], check=True)

print(f"Pushed Docker image to ECS repository: {ecs_repository_uri}")


AttributeError: <botocore.errorfactory.ECSExceptions object at 0x7f85218e1640> object has no attribute RepositoryAlreadyExistsException. Valid exceptions are: AccessDeniedException, AttributeLimitExceededException, BlockedException, ClientException, ClusterContainsContainerInstancesException, ClusterContainsServicesException, ClusterContainsTasksException, ClusterNotFoundException, InvalidParameterException, LimitExceededException, MissingVersionException, NamespaceNotFoundException, NoUpdateAvailableException, PlatformTaskDefinitionIncompatibilityException, PlatformUnknownException, ResourceInUseException, ResourceNotFoundException, ServerException, ServiceNotActiveException, ServiceNotFoundException, TargetNotConnectedException, TargetNotFoundException, TaskSetNotFoundException, UnsupportedFeatureException, UpdateInProgressException

## create infrastructure

In [2]:
ec2 = create_aws_service(service=EC2_CONFIG['service_name'],
                             region=REGION
                             )
iam = create_aws_service(IAM_CONFIG['service_name'],
                             region=REGION
)

code_deploy = create_aws_service(CODE_DEPLOY_CONFIG['service_name'],
                             region=REGION
)

s3 = create_aws_service(S3_CONFIG['service_name'],
                             region=REGION
)
sts = create_aws_service(STS_CONFIG['service_name'],
                             region=REGION
)
ecs = create_aws_service(ECS_CONFIG['service_name'],
                             region=REGION
)

Service ec2 is created
Service iam is created
Service codedeploy is created
Service s3 is created
Service sts is created
Service ecs is created
